<a href="https://colab.research.google.com/github/JockWang/colab/blob/master/%E4%BC%81%E4%B8%9A%E5%A4%B1%E4%BF%A1%E9%A2%84%E6%B5%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!pip install catboost

     |████████████████████████████████| 63.0MB 153kB/s 


In [0]:
import pandas as pd
import numpy as np
import catboost as cat
import lightgbm as lgb
import xgboost as xgb
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV, train_test_split, StratifiedKFold
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, MinMaxScaler, OneHotEncoder
import time

#显示所有列
pd.set_option('display.max_columns', None)
#显示所有行
pd.set_option('display.max_rows', None)

In [0]:
label_data = pd.read_csv('/content/drive/My Drive/比赛/DC_shixin/shixin/train_label.csv')
train_data = pd.read_csv('/content/drive/My Drive/比赛/DC_shixin/shixin/train.csv')
test_data = pd.read_csv('/content/drive/My Drive/比赛/DC_shixin/shixin/test.csv')
submit = pd.read_csv('/content/drive/My Drive/比赛/DC_shixin/shixin/submission.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
data = train_data.append(test_data)
data.head(3)

,ID,企业类型,经营期限至,登记机关,企业状态,邮政编码,投资总额,注册资本,核准日期,行业代码,注销时间,经营期限自,成立日期,行业门类,企业类别,管辖机关,经营范围,增值税,企业所得税,印花税,教育费,城建税,递延收益_年初数,长期负债合计_年初数,长期借款_年初数,长期应付款_年初数,短期借款_年初数,递延所得税负债_年初数,非流动负债合计_年初数,负债合计_年初数,负债和所有者权益总计_年初数,交易性金融负债_年初数,流动负债合计_年初数,其他非流动负债_年初数,其他负债（或长期负债）_年初数,其他流动负债_年初数,其他应付款_年初数,其他应交款_年初数,实收资本（股本）_年初数,实收资本（或股本）净额_年初数,所有者权益合计_年初数,未分配利润_年初数,小企业_应付利润_年初数,应付福利费_年初数,应付股利_年初数,应付利息_年初数,应付票据_年初数,应付职工薪酬_年初数,应付债券_年初数,应付账款_年初数,预计负债_年初数,应交税费_年初数,一年内到期的非流动负债_年初数,预提费用_年初数,预收款项_年初数,盈余公积_年初数,资本公积_年初数,专项应付款_年初数,递延收益_年末数,长期负债合计_年末数,长期借款_年末数,长期应付款_年末数,短期借款_年末数,递延所得税负债_年末数,非流动负债合计_年末数,负债合计_年末数,负债和所有者权益总计_年末数,交易性金融负债_年末数,流动负债合计_年末数,其他非流动负债_年末数,其他负债（或长期负债）_年末数,其他流动负债_年末数,其他应付款_年末数,其他应交款_年末数,实收资本（股本）_年末数,实收资本（或股本）净额_年末数,所有者权益合计_年末数,未分配利润_年末数,小企业_应付利润_年末数,应付福利费_年末数,应付股利_年末数,应付利息_年末数,应付票据_年末数,应付职工薪酬_年末数,应付债券_年末数,应付账款_年末数,预计负债_年末数,应交税费_年末数,一年内到期的非流动负债_年末数,预提费用_年末数,预收款项_年末数,盈余公积_年末数,资本公积_年末数,专项应付款_年末数,货币资金_年初数,存货_年初数,存货中的原材料_年初数,存货中的周转材料_年初数,长期待摊费用_年初数,长期股权投资_年初数,长期投资合计_年初数,长期应收款_年初数,持有至到期投资_年初数,短期投资_年初数,待摊费用_年初数,递延所得税资产_年初数,非流动资产合计_年初数,工程物资_年初数,固定资产合计_年初数,固定资产净额_年初数,固定资产净值_年初数,固定资产清理_年初数,固定资产原价_年初数,交易性金融资产_年初数,库存商品(产成品)_年初数,开发支出_年初数,可供出售金融资产_年初数,流动资产合计_年初数,其他非流动资产_年初数,其他流动资产_年初数,其他应收款_年初数,生产性生物资产_年初数,商誉_年初数,投资性房地产_年初数,无形资产_年初数,无形资产及其他资产合计_年初数,应收账款_年初数,一年内到期的非流动资产_年初数,油气资产_年初数,应收补贴款_年初数,应收股利_年初数,应收利息_年初数,应收票据_年初数,预付款项_年初数,资产总计_年初数,在建工程_年初数,货币资金_年末数,存货_年末数,存货中的原材料_年末数,存货中的周转材料_年末数,长期待摊费用_年末数,长期股权投资_年末数,长期投资合计_年末数,长期应收款_年末数,持有至到期投资_年末数,短期投资_年末数,待摊费用_年末数,递延所得税资产_年末数,非流动资产合计_年末数,工程物资_年末数,固定资产合计_年末数,固定资产净额_年末数,固定资产净值_年末数,固定资产清理_年末数,固定资产原价_年末数,交易性金融资产_年末数,库存商品(产成品)_年末数,开发支出_年末数,可供出售金融资产_年末数,流动资产合计_年末数,其他非流动资产_年末数,其他流动资产_年末数,其他应收款_年末数,生产性生物资产_年末数,商誉_年末数,投资性房地产_年末数,无形资产_年末数,无形资产及其他资产合计_年末数,应收账款_年末数,一年内到期的非流动资产_年末数,油气资产_年末数,应收补贴款_年末数,应收股利_年末数,应收利息_年末数,应收票据_年末数,预付款项_年末数,资产总计_年末数,在建工程_年末数
0,1,9.0,NaN,1.0,0.0,266300,19.110000,19.110000,00:00.0,5154.0,NaN,00:00.0,00:00.0,1.0,0.0,1.0,"[1412, 1024, 21375, 15023, 18526, 18789, 17022...",106.03965,0.331552,1.247218,3.181189,7.422776,0.0,0.0,25.110484,0.0,0.0,0.0,25.110484,797.386637,844.162546,0.0,772.276153,0.0,0.0,0.0,106.078436,0.0,50.220972,0.0,46.77591,-3.445063,0.0,0.0,0.0,0.0,125.552431,0.0,0.0,520.839979,0.0,19.805307,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,585.013852,632.032182,0.0,585.013852,0.0,0.0,0.0,83.123866,0.0,50.220972,0.0,47.01833,-3.202643,0.0,0.0,0.0,0.0,0.0,0.0,0.0,489.934244,0.0,11.955743,0.0,0.0,0.0,0.0,0.0,0.0,24.579246,462.290647,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,135.081011,0.0,135.081011,0.0,0.0,0.0,368.329907,0.0,0.0,0.0,0.0,709.081535,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,215.371727,0.0,0.0,0.0,0.0,0.0,2.647788,4.192126,844.162546,0.0,31.416284,229.147518,0.0,0.0,0.0,0.0,0.0,0.0,0.0,80.128884,0.0,0.0,99.813781,0.0,99.813781,0.0,0.0,0.0,368.329907,0.0,0.0,0.0,0.0,532.218401,0.0,0.0,20.088389,0.0,0.0,0.0,0.0,0.0,142.310005,0.0,0.0,0.0,0.0,0.0,28.122902,1.004419,632.032182,0.0
1,3,11.0,NaN,5.0,0.0,266000,29.154194,19.210446,00:00.0,7214.0,NaN,31:40.0,31:40.0,0.0,2.0,3.0,"[3671, 24856, 13525, 21079, 13219, 13694, 1442...",39.17560,51.244655,0.014363,1.175269,2.742293,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,9,0.0,NaN,3.0,0.0,266000,NaN,19.120045,13:31.0,2110.0,NaN,13:31.0,13:31.0,2.0,0.0,4.0,"[3671, 14394, 23662, 11227, 19205, 15633, 5344...",0.00000,0.000000,0.541067,4.637491,10.060346,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

In [0]:
data.shape

(23944, 178)

In [0]:
def buScope500(x):
  scope = list(map(int, set(x[1:-1].split(', '))))
  layers = []
  for s in scope:
    layers.append(s//500)
  return len(set(layers))
def buScope1000(x):
  scope = list(map(int, set(x[1:-1].split(', '))))
  layers = []
  for s in scope:
    layers.append(s//1000)
  return len(set(layers))
def buScope5000(x):
  scope = list(map(int, set(x[1:-1].split(', '))))
  layers = []
  for s in scope:
    layers.append(s//5000)
  return len(set(layers))

In [0]:
def cleanData(train_data):    
  train = train_data.copy()
  numbers = []
  # 信息确实率
  train['信息缺失率'] = train.isna().sum(axis=1)/178
  for col in list(train.columns):
    if col in ['经营期限至','核准日期','注销时间','经营期限自','成立日期']:
      train[col] = train[col].apply(lambda x: 0 if str(x) == 'nan' else int(x.split('.')[0].split(':')[0])*60+int(x.split('.')[0].split(':')[1]))
    elif col in ['登记机关']:# 类别
      train[col] = train[col].apply(lambda x: 12 if str(x) == 'nan' else int(x))
    elif col in ['企业类型']:# 类别
      train[col] = train[col].apply(lambda x: 84 if str(x) == 'nan' else int(x))
    elif col in ['企业状态']:# 类别
      train[col] = train[col].apply(lambda x: 4 if str(x) == 'nan' else int(x))
    elif col in ['邮政编码']:# 类别
      train[col] = train[col].apply(lambda x: 266000 if len(str(x)) < 6 else int(str(x)[:6].replace('.','')))
    elif col in ['投资总额','注册资本','企业所得税','印花税','教育费','城建税']:
      train[col] = train[col].apply(lambda x: 0 if str(x) == 'nan' else float(x))
    elif col in ['行业代码']:# 类别
      train[col] = train[col].apply(lambda x: 0 if str(x) == 'nan' else int(x))
    elif col in ['行业门类']:# 类别
      train[col] = train[col].apply(lambda x: 19 if str(x) == 'nan' else int(x))
    elif col in ['企业类别']:# 类别
      train[col] = train[col].apply(lambda x: 4 if str(x) == 'nan' else int(x))
    elif col in ['管辖机关']:# 类别
      train[col] = train[col].apply(lambda x: 13 if str(x) == 'nan' else int(x))
    elif col in ['经营范围']:
      train['经营范围_Set'] = train['经营范围'].apply(lambda x: 0 if str(x) == 'nan' else len(set(x[1:-1].split(', '))))
      train['经营范围大小'] = train[col].apply(lambda x: 0 if str(x) == 'nan' else len(x[1:-1].split(', ')))
      train['经营范围粗粒度_500'] = train['经营范围'].apply(buScope500)
      train['经营范围粗粒度_1000'] = train['经营范围'].apply(buScope1000)
      train['经营范围粗粒度_5000'] = train['经营范围'].apply(buScope5000)
    elif '年初数' in col or '年末数' in col:
      train[col] = train[col].apply(lambda x: 0.0 if str(x) == 'nan' else float(x))
      name = col[:-4]
      if name not in numbers:
        numbers.append(name)
  for num in numbers:
    train[num] = train[num+'_年末数']-train[num+'_年初数']

  train['所得税比增值税'] = train['企业所得税']/train['增值税']

  train['经营时长'] = train['经营期限自'] - train['经营期限至']
  
  train = train.drop(columns=['经营范围'])

  return train

In [0]:
train = cleanData(train_data)
test = cleanData(test_data)

#### 训练CatBoost

In [0]:
cat_features = ['ID','登记机关','企业类型','企业状态','邮政编码','行业代码','行业门类','企业类别','管辖机关']

In [0]:
def KfoldResult(train=train,label=label_data.values[:,1],test=test,cat_features=cat_features):
    fold = StratifiedKFold(n_splits=10)
    results = []
    i = 0
    for train_index, valid_index in fold.split(train,label_data.values[:,1]):
        print('第',str(i),'个Fold')
        i += 1
        train_x,valid_x,train_y,valid_y = train.iloc[train_index],train.iloc[valid_index],label[train_index],label[valid_index]
        eval_set = cat.Pool(data=valid_x,label=valid_y,cat_features=cat_features)
        model = cat.CatBoostClassifier(iterations=5000, learning_rate=0.1, depth=10, loss_function='Logloss', eval_metric='AUC')
        model.fit(train_x,train_y,use_best_model=True,eval_set=eval_set,cat_features=cat_features,early_stopping_rounds=20)
        pred = model.predict_proba(test)[:,1]
        results.append(pred)

    results = np.array(results).mean(axis=0)
    return results

In [0]:
results = KfoldResult()

第 0 个Fold
0:	test: 0.8182702	best: 0.8182702 (0)	total: 234ms	remaining: 19m 30s
1:	test: 0.8637183	best: 0.8637183 (1)	total: 463ms	remaining: 19m 16s
2:	test: 0.8661755	best: 0.8661755 (2)	total: 499ms	remaining: 13m 50s
3:	test: 0.8703766	best: 0.8703766 (3)	total: 728ms	remaining: 15m 9s
4:	test: 0.8750950	best: 0.8750950 (4)	total: 963ms	remaining: 16m 1s
5:	test: 0.8757534	best: 0.8757534 (5)	total: 992ms	remaining: 13m 45s
6:	test: 0.8818160	best: 0.8818160 (6)	total: 1.22s	remaining: 14m 32s
7:	test: 0.8839715	best: 0.8839715 (7)	total: 1.31s	remaining: 13m 36s
8:	test: 0.8806678	best: 0.8839715 (7)	total: 1.54s	remaining: 14m 15s
9:	test: 0.8806129	best: 0.8839715 (7)	total: 1.78s	remaining: 14m 47s
10:	test: 0.8837716	best: 0.8839715 (7)	total: 2s	remaining: 15m 9s
11:	test: 0.8858369	best: 0.8858369 (11)	total: 2.25s	remaining: 15m 34s
12:	test: 0.8846534	best: 0.8858369 (11)	total: 2.48s	remaining: 15m 52s
13:	test: 0.8845123	best: 0.8858369 (11)	total: 2.51s	remaining: 14m

## 训练LGB

In [0]:
x_train,x_valid,y_train,y_valid = train_test_split(train,label_data.values[:,1],test_size=0.1,stratify=label_data.values[:,1])
train_d = lgb.Dataset(x_train,y_train)
valid_d = lgb.Dataset(x_valid,y_valid)

In [0]:
lgb_params = {
  'objective': 'binary',
  # 'num_class': 7,
  'learning_rate': 0.1,
  'num_leaves': 32,
  'bagging_fraction': 1,
  'feature_fraction': .8,
  'bagging_freq': 0,
  'lambda_l1': 0.,
  'lambda_l2': 0.1,
  'metric': 'auc',
}
model = lgb.train(lgb_params,train_set=train_d,valid_sets=valid_d,num_boost_round=1000,early_stopping_rounds=20)

[1]	valid_0's auc: 0.885664
Training until validation scores don't improve for 20 rounds.
[2]	valid_0's auc: 0.894263
[3]	valid_0's auc: 0.914371
[4]	valid_0's auc: 0.913297
[5]	valid_0's auc: 0.919377
[6]	valid_0's auc: 0.921445
[7]	valid_0's auc: 0.923369
[8]	valid_0's auc: 0.923199
[9]	valid_0's auc: 0.923623
[10]	valid_0's auc: 0.923548
[11]	valid_0's auc: 0.924043
[12]	valid_0's auc: 0.92516
[13]	valid_0's auc: 0.925215
[14]	valid_0's auc: 0.925746
[15]	valid_0's auc: 0.925663
[16]	valid_0's auc: 0.925088
[17]	valid_0's auc: 0.926094
[18]	valid_0's auc: 0.926728
[19]	valid_0's auc: 0.93
[20]	valid_0's auc: 0.929702
[21]	valid_0's auc: 0.92992
[22]	valid_0's auc: 0.929893
[23]	valid_0's auc: 0.929001
[24]	valid_0's auc: 0.929069
[25]	valid_0's auc: 0.929409
[26]	valid_0's auc: 0.93259
[27]	valid_0's auc: 0.932709
[28]	valid_0's auc: 0.933097
[29]	valid_0's auc: 0.934471
[30]	valid_0's auc: 0.934241
[31]	valid_0's auc: 0.933679
[32]	valid_0's auc: 0.932919
[33]	valid_0's auc: 0.9328

In [0]:
def LGBKfoldResult(train=train,label=label_data.values[:,1],test=test,params=lgb_params,K=20):
  fold = StratifiedKFold(n_splits=K,shuffle=True)
  preds = []
  i = 0
  for train_index, valid_index in fold.split(train,label):
    print('第',str(i),'个Fold')
    i += 1
    train_x,valid_x,train_y,valid_y = train.iloc[train_index],train.iloc[valid_index],label[train_index],label[valid_index]
    lgb_train = lgb.Dataset(x_train,y_train)
    lgb_valid = lgb.Dataset(x_valid,y_valid)
    model = lgb.train(params,train_set=lgb_train,valid_sets=lgb_valid,num_boost_round=1000,early_stopping_rounds=20)
    pred = model.predict(test)
    print(pred)
    preds.append(pred)
  preds = np.array(preds)
  return preds

In [0]:
preds = LGBKfoldResult().mean(axis=0)

第 0 个Fold
[1]	valid_0's auc: 0.885664
Training until validation scores don't improve for 20 rounds.
[2]	valid_0's auc: 0.894263
[3]	valid_0's auc: 0.914371
[4]	valid_0's auc: 0.913297
[5]	valid_0's auc: 0.919377
[6]	valid_0's auc: 0.921445
[7]	valid_0's auc: 0.923369
[8]	valid_0's auc: 0.923199
[9]	valid_0's auc: 0.923623
[10]	valid_0's auc: 0.923548
[11]	valid_0's auc: 0.924043
[12]	valid_0's auc: 0.92516
[13]	valid_0's auc: 0.925215
[14]	valid_0's auc: 0.925746
[15]	valid_0's auc: 0.925663
[16]	valid_0's auc: 0.925088
[17]	valid_0's auc: 0.926094
[18]	valid_0's auc: 0.926728
[19]	valid_0's auc: 0.93
[20]	valid_0's auc: 0.929702
[21]	valid_0's auc: 0.92992
[22]	valid_0's auc: 0.929893
[23]	valid_0's auc: 0.929001
[24]	valid_0's auc: 0.929069
[25]	valid_0's auc: 0.929409
[26]	valid_0's auc: 0.93259
[27]	valid_0's auc: 0.932709
[28]	valid_0's auc: 0.933097
[29]	valid_0's auc: 0.934471
[30]	valid_0's auc: 0.934241
[31]	valid_0's auc: 0.933679
[32]	valid_0's auc: 0.932919
[33]	valid_0's a

KeyboardInterrupt: ignored

In [0]:
submit.Label = preds
submit

,ID,Label
0,0,0.014458
1,2,0.122772
2,4,0.016637
3,5,0.014997
4,6,0.014918
...,...,...
9573,23932,0.015283
9574,23935,0.364532
9575,23936,0.014409
9576,23938,0.018032


In [0]:
submit.to_csv('submit.csv',index=False)